In [1]:
import os
import sys

sys.dont_write_bytecode = True
os.environ["NUMBA_DISABLE_JIT"] = "1"

import numpy as np
import pandas as pd

from quantfreedom.evaluators.evaluators import is_below
from quantfreedom.indicators.talib_ind import from_talib
from quantfreedom.testing_stuff.base_testing import backtest_df_only_testing
from quantfreedom.testing_stuff.enums_testing import (
    CandleBody,
    LeverageMode,
    OrderSettingsArrays,
    OrderType,
    SizeType,
    StaticVariables,
)

np.set_printoptions(formatter={"float_kind": "{:.2f}".format})
pd.options.display.float_format = "{:,.2f}".format

price_data = pd.read_hdf("data/prices.hd5")
rsi_ind = from_talib(
    func_name="rsi",
    price_data=price_data,
    cart_product=False,
    combos=False,
    timeperiod=[15, 30],
)
entries = is_below(
    want_to_evaluate=rsi_ind,
    user_args=[40, 50],
    candle_ohlc="close",
)

In [2]:
static_vars = StaticVariables(
    equity=1000.,
    lev_mode=LeverageMode.LeastFreeCashUsed,
    order_type=OrderType.LongEntry,
    size_type=SizeType.RiskPercentOfAccount,
)
order_settings = OrderSettingsArrays(
    max_equity_risk_pct=6.,
    risk_reward=5,
    size_pct=1.,
    sl_based_on=CandleBody.low,
    sl_based_on_lookback=30,
    sl_based_on_add_pct=.2,
)
backtest_df_only_testing(
    price_data=price_data,
    entries=entries,
    static_variables_tuple=static_vars,
    order_settings_arrays_tuple=order_settings,
)

Checking static variables for errors or conflicts.
Turning all variables into arrays.
Checking arrays for errors or conflicts ... the backtest will begin shortly, please hold.
Creating cartesian product ... after this the backtest will start, I promise :).

Starting the backtest now ... and also here are some stats for your backtest.

Total symbols: 5
Total indicator settings per symbol: 4
Total indicator settings to test: 20
Total order settings per symbol: 1
Total order settings to test: 5
Total candles per symbol: 9,408
Total candles to test: 188,160

Total combinations to test: 20


(     symbol  entries_col  or_set  total_trades  gains_pct  win_rate  \
 3   BTCUSDT            3       0        117.00     123.40     29.91   
 8    ETHUSD            8       0         87.00     198.54     28.74   
 11   ETHUSD           11       0         84.00     171.69     30.95   
 9    ETHUSD            9       0         73.00      84.71     31.51   
 1   BTCUSDT            1       0         79.00      93.62     31.65   
 0   BTCUSDT            0       0        131.00     118.27     27.48   
 15  LTCUSDT           15       0        117.00     125.36     27.35   
 12  LTCUSDT           12       0        118.00      76.13     23.73   
 10   ETHUSD           10       0         64.00      68.97     26.56   
 13  LTCUSDT           13       0        107.00     124.44     28.97   
 5    EOSUSD            5       0         32.00      66.26     31.25   
 4    EOSUSD            4       0         37.00      49.08     24.32   
 7    EOSUSD            7       0         35.00      52.94     2